### Import packages

In [1]:
import imlreliability
import pandas as pd
import numpy as np

/Users/melinda/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Dimension Reduction 

#### Load data

In [2]:
from sklearn.model_selection import train_test_split
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data', header=None)
data=data.dropna()
y=(data[1])
x = data.drop(columns=[0,1]).to_numpy()
K=len(set(y))

### Noise Addition


####  + Clustering

In [7]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
esti=PCA()

model = imlreliability.dimension_reduction.dimension_reduction(x,estimator=esti,K=len(set(y)),
                 label=y,
                 perturbation = 'noise',
                rank=2,
                 sigma=1,
                 noise_type='normal',
                 n_repeat=3,
                    rand_index=1,
                    verbose=True)

model.fit()
model.consistency_clustering('WDBC','PCA',KMeans(n_clusters=4),'KM')
print(model.results)



Iter:  0
Iter:  1
Iter:  2
noise
noise
noise
       data method perturbation clustering   noise  sigma  rank  \
0  Spambase    PCA        noise         KM  normal      1     2   
1  Spambase    PCA        noise         KM  normal      1     2   
2  Spambase    PCA        noise         KM  normal      1     2   
3  Spambase    PCA        noise         KM  normal      1     2   

                criteria  Accuracy  Consistency  
0                    ARI  0.328667     0.488333  
1  Fowlkes Mallows Score  0.598667     0.620000  
2     Mutual Information  0.402667     0.506667  
3        V Measure Score  0.404333     0.509667  


#### +KNN

In [8]:
## Nearest neighbor 

model.consistency_knn('WDBC','PCA')
# print(model.consistency_knn_mean)
print(model.aucc)

       data method   noise  sigma criteria  rank  Consistency
0  Spambase    PCA  normal      1  Jaccard     2     0.567629


#### without labels 

In [6]:
model2 = imlreliability.dimension_reduction.dimension_reduction(x,estimator=esti,K=len(set(y)),
                 label=None,
                 perturbation = 'noise',
                rank=2,
                 sigma=1,
                 noise_type='normal',
                 n_repeat=3,
                    rand_index=1,
                    verbose=True)

model2.fit()
model2.consistency_clustering('WDBC','PCA',KMeans(n_clusters=4),'KM')
print(model2.results)

       data method perturbation clustering   noise  sigma  rank  \
0  Spambase    PCA        noise         KM  normal      1     2   
1  Spambase    PCA        noise         KM  normal      1     2   
2  Spambase    PCA        noise         KM  normal      1     2   
3  Spambase    PCA        noise         KM  normal      1     2   

                criteria  Consistency  Accuracy  
0                    ARI     0.488333       NaN  
1  Fowlkes Mallows Score     0.620000       NaN  
2     Mutual Information     0.506667       NaN  
3        V Measure Score     0.509667       NaN  


### Data spliting

In [3]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
esti=PCA()

model = imlreliability.dimension_reduction.dimension_reduction(x,estimator=esti,K=len(set(y)),
                 label=y,
                 perturbation = 'split',
                rank=2,
                 n_repeat=3,
                    rand_index=1,
                    verbose=True)

model.fit()
model.consistency_clustering('WDBC','PCA',KMeans(n_clusters=4),'KM')
print(model.results)



Iter:  0
Iter:  1
Iter:  2
       data method perturbation clustering   noise  sigma  rank  \
0  Spambase    PCA        split         KM  normal      1     2   
1  Spambase    PCA        split         KM  normal      1     2   
2  Spambase    PCA        split         KM  normal      1     2   
3  Spambase    PCA        split         KM  normal      1     2   

                criteria  Accuracy  Consistency  
0                    ARI  0.364667     0.749667  
1  Fowlkes Mallows Score  0.626000     0.816000  
2     Mutual Information  0.442000     0.761000  
3        V Measure Score  0.444000     0.764333  
